In [ ]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
%config InlineBackend.figure_format='retina'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img

# pretrained model
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

from tensorflow.keras.preprocessing.image import ImageDataGenerator

Prepare data for the model

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_ds = train_gen.flow_from_directory('./clothing-dataset-small/train/', 
                              target_size=(150, 150), 
                                batch_size=32)
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_ds = val_gen.flow_from_directory('./clothing-dataset-small/validation/', 
                              target_size=(150, 150), 
                                batch_size=32,
                                shuffle=False) 

In [ ]:
base_model = Xception(
                weights='imagenet', 
                include_top=False,
                input_shape=(150, 150, 3)
)
# use conv.layers of Xception
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(150, 150, 3))
base = base_model(inputs, training = False) 

# add pooling
pooling = keras.layers.GlobalAveragePooling2D()
vectors = pooling(base)
outputs = keras.layers.Dense(10)(vectors)

In [ ]:
learning_rate=0.001

In [ ]:
# Function to create model
def make_model(learning_rate=0.01):
    #################
    # # base (pre-trained) model
    # base_model = Xception(
    #                 weights='imagenet', 
    #                 include_top=False,
    #                 input_shape=(150, 150, 3)
    # )
    # # use conv.layers of Xception
    # base_model.trainable = False
    
    # #########################################
    
    # inputs = keras.Input(shape=(150, 150, 3))
    # base = base_model(inputs, training = False) 

    # # add pooling
    # pooling = keras.layers.GlobalAveragePooling2D()
    # vectors = pooling(base)
    # outputs = keras.layers.Dense(10)(vectors) # outputs of conv.layers
    ####################
    
    model = keras.Model(inputs, outputs)
    
    #########################################
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['accuracy'])
    
    return model

### Checkpoints

In [ ]:
model.save_weights('model_v1.h5', save_format='h5')

In [ ]:

checkpoint = [
    keras.callbacks.ModelCheckpoint(
        "xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5",
        monitor="val_accuracy",
        save_best_only=True,
        mode='max'
)
]


In [ ]:
learning_rate = 0.001

model = make_model(learning_rate=learning_rate)

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=[checkpoint]
)

### Adding more layers

In [ ]:
# Function to create model
def make_model(learning_rate=0.01, size_inner=10):
    #################
    # base (pre-trained) model
    base_model = Xception(
                    weights='imagenet', 
                    include_top=False,
                    input_shape=(150, 150, 3)
    )
    # use conv.layers of Xception
    base_model.trainable = False
    
    #########################################
    
    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training = False) 

    # add pooling
    pooling = keras.layers.GlobalAveragePooling2D()
    vectors = pooling(base)
    # add inner layer
    inner = keras.layers.Dense(size_inner, activation="relu")(vectors)
    # replace vectors with inner layer's outputs
    
    outputs = keras.layers.Dense(10)(inner) # outputs of conv.layers
    ####################
    
    model = keras.Model(inputs, outputs)
    
    #########################################
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    # Compile the model
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=['accuracy'])
    
    return model

In [ ]:
scores = {}
lr = 0.001
# List of learning rates
size_inner = [10, 100, 1000]

for size in size_inner:
    print(lr)
    
    model = make_model(learning_rate=lr, size_inner=size)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[size] = history.history
    
    print()
    print()

In [ ]:
for size, hist in scores.items():
    plt.plot(hist["val_accuracy"], label=("val=%s" % size))
plt.xticks(np.arange(10))
plt.legend()
plt.show()

### Regularization and Dropout